In [1]:
!pip install openai pinecone-client sentence-transformers


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 386.9/386.9 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.8/244.8 kB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 255.2/255.2 kB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.2/325.2 kB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.4 MB/s eta 0:00:00


In [3]:
import openai
import pinecone
from sentence_transformers import SentenceTransformer
import numpy as np


In [4]:
openai.api_key = "YOUR_OPENAI_API_KEY"


In [6]:
pinecone.init(api_key="YOUR_PINECONE_API_KEY", environment="us-west1-gcp")

# Check if the index exists, if not, create one
index_name = 'business-qa'
if index_name not in pinecone.list_indexes():
    pinecone.create_index(name=index_name, dimension=768, metric="cosine")
index = pinecone.Index(index_name)


In [8]:
model = SentenceTransformer('all-MiniLM-L6-v2')

# Example business knowledge base
documents = [
    "Our company provides cloud solutions for businesses.",
    "We offer 24/7 customer support to our clients.",
    "The annual subscription for our product is $500.",
]

# Generate embeddings
embeddings = model.encode(documents).tolist()

# Upsert embeddings into Pinecone index
ids = [f"doc_{i}" for i in range(len(documents))]
index.upsert(vectors=list(zip(ids, embeddings)))


In [9]:
def retrieve(query, top_k=3):
    query_embedding = model.encode([query]).tolist()
    result = index.query(queries=query_embedding, top_k=top_k)
    return result


In [10]:
def generate_answer(query, context):
    prompt = f"Answer the question based on the context below:\n\nContext: {context}\n\nQuestion: {query}\nAnswer:"
    response = openai.Completion.create(
        engine="text-davinci-003",
        prompt=prompt,
        max_tokens=150
    )
    return response['choices'][0]['text'].strip()


In [11]:
def qa_bot(query):
    # Step 1: Retrieve relevant documents
    retrieved_docs = retrieve(query)

    # Step 2: Concatenate the retrieved documents as context
    context = "\n".join([doc['metadata']['text'] for doc in retrieved_docs['matches']])

    # Step 3: Generate the answer using OpenAI GPT model
    answer = generate_answer(query, context)

    return answer


In [ ]:
query = "What is the annual subscription for the product?"
answer = qa_bot(query)
print(f"Answer: {answer}")
